### Лекция 4. Классы продолжение

<br />

##### Спецификация вызова конструктора базового класса

Базовый класс может иметь несколько конструкторов, а наследник может выбирать, какой именно из конструкторов он хочет вызвать. Это определяется в списке инициализации конструктора

In [ ]:
class Animal
{
public:
    Animal();
    Animal(const std::string& a_name);
    Animal(const std::string& a_name, unsigned a_age);

private:
    std::string name;
    unsigned age;
};


class Cat : public Animal
{
public:
    // # неявно вызывает Animal()
    Cat() {}    
    // # эквивалентно записи:
    // # Cat() : Animal() {}
    
    // # явный вызов конкретного конструктора Animal
    Cat(const std::string& name)
        : Animal(name)  // # сначала вызываем конструктор базового класса
        , color(White)  // # затем конструкторы членов в порядке их определения
    {
    }
    
private:
    Color color = Black;  // # значение для инициализации по умолчанию
};

<br />

##### Множественное наследование, ромбовидное наследование

Обратить внимание на порядок конструирования, вызова деструкторов и способа разрешения конфликтов по именам

In [ ]:
class Animal
{
    ...
};

class JumpingCreature
{
public:
    void jump();
};

class CryingCreature
{
public:
    void cry();    
};


class Cat : public Animal
          , public JumpingCreature
          , public CryingCreature
{
public:
    void play()
    {
        jump();
        jump();
        cry();
        jump();
        
        // # явное указание метода,
        // #
        // # потребуется, если метод jump()
        // # есть и у JumpingCreature, и у Animal
        JumpingCreature::jump();
        CryingCreature::cry();
    }
};

<img src="classes_diagram_cat.png" height=500px width=500px />

<br />

Граф наследования ацикличен и однонаправлен... но ничто не мешает сделать в нём ромб. Такая конструкция выглядит необычной, но, тем не менее, используется даже в стандартной библиотеке:

![](std-basic_iostream-inheritance.png)

https://en.cppreference.com/w/cpp/io/basic_iostream

<br />

##### Layout класса

Рассмотрим как устроено расположение класса в памяти. Пойдём от простого к сложному.

In [ ]:
struct Point
{
    float x;  // # size = 4, alignment = 4
    float y;  // # size = 4, alignment = 4
};  // # size = 8, alignment = 4

![](layout_point.jpg)

In [ ]:
class A
{
    float x;          // # size = 4, alignment = 4
    std::uint64_t y;  // # size = 8, alignment = 8  
};  // # size = ???, alignment = ???

![](layout_padding.jpg)

Более подробно про alignment:
https://en.cppreference.com/w/c/language/object

In [ ]:
class A
{
    float x;          // # size = 4, alignment = 4
    std::uint64_t y;  // # size = 8, alignment = 8
    float z;          // # size = 4, alignment = 4
};  // # size = ???, alignment = ???

![](layout_padding_2.jpg)

In [ ]:
class A
{
    std::uint64_t y;  // # size = 8, alignment = 8
    float x;          // # size = 4, alignment = 4
    float z;          // # size = 4, alignment = 4
};  // # size = ???, alignment = ???

![](layout_padding_3.jpg)

<br />

Рассмотрим layout класса в памяти в случае наследования:

In [ ]:
struct P
{
    float x;  // # size = 4, alignment = 4
    float y;  // # size = 4, alignment = 4
};  // # size = 8, alignment = 4

struct WP : P
{
    float w;  // # size = 4, alignment = 4
};  // # size = ???, alignment = ???

![](layout_inheritance.jpg)

Рассмотрим layout класса в памяти в случае множественного наследования:

In [ ]:
class CBase1 { ... };
class CBase2 { ... };

class CDerived : public CBase1
               , public CBase2
{ ... };

![](layout_multiple_inheritance.jpg)

<br />

Как быть с виртуальными методами и функциями:

In [ ]:
class Base
{
public:
    virtual void say_hello();
    virtual void say_goodbye();
    
private:
    ...
};

class Dervied : public Base
{
public:
    void say_hello() override;
    void say_goodbye() override;
    
private:
    ...
};

![](layout_vtable_2.jpg)

Показать как ведёт себя преобразование указателей для такого лаяута (и почему с reinterpert_cast лучше не переборщить).

Открыть на godbolt.org пример и показать как происходит сравнение указателей, закомментировать виртуальные функции - посмотреть что происходит.

In [ ]:
struct Base1
{
    // virtual void say_hello() = 0;
    float x;
};

struct Base2
{
    float x;
};

struct Derived : public Base1
               , public Base2
{
    // virtual void say_hello() = 0;
    float y;
};

bool equal1(Base1* lhs, Derived *rhs)
{
    return lhs == rhs;
}

bool equal2(Base2* lhs, Derived *rhs)
{
    return lhs == rhs;
}

<br />

##### Состояние класса, const + mutable

Что считать состоянием класса?
* значения всех его полей
* логическое состояние класса (набор элементов в векторе, а не детали типа `capacity`)

С точки зрения компилятора - набор всех его полей, не помеченных словом `mutable`

In [ ]:
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // # аттрибут const означает, что метод не будет
    // # менять полей класса (и компилятор это проверит!)
    // #
    // # внутри const-методов поля имеют тип:
    // # const std::string name;
    // # const unsigned age;
    bool is_too_young() const
    {
        return age <= 3;
    }
    
    // # метод не меняет полей класса, но программист
    // # забыл поставить const
    void say_hello()
    {
        std::cout << "hello";
    }
    
    // # метод не меняет полей класса
    void say_hello_twice() const
    {
        say_hello();  // # COMPILE-TIME ERROR
        say_hello();
        
        name = "unk";  // # COMPILE-TIME ERROR
    }
    
    // # отсутствие const - метод МОЖЕТ менять поля
    void happy_birthday()
    {
        age += 1; // # OK
    }   
};

Рассмотрим учебный пример - отображение, которое хранит ответ на последний запрос как кеширование.

In [ ]:
class Resources
{
public:
    void add(int id, const std::string& resource);
    
    const std::string& get(int id) const;
    
private:
    // # хранилище ресурсов
    std::map<int, std::string> id_to_resource;
    
    // # закешированный последний запрошенный ресурс
    mutable int last_id = -1;
    mutable std::string last_resource;
};

void Resources::add(const int id, const std::string& resource)
{
    id_to_resource[id] = resource;
}

const std::string& Resources::get(const int id) const
{
    if (id == last_id)
        return last_resource;
    
    auto it = id_to_resource.find(id);  // # |find| - const-метод
    
    last_resource = it->second;  // # OK, т.к. mutable
    last_id = id;                // # OK, т.к. mutable
    
    return last_resource;
}

Примечание: это исключительно ученический пример кеширования для демонстрации понятий `mutable` и состояния объекта, никогда так не реализуйте кеширование.

<br />

##### Операторы

https://en.cppreference.com/w/cpp/language/operators

В виде свободных функций

In [ ]:
struct Vector3D
{
    double x;
    double y;
    double z;
};

// # v1 + v2
Vector3D operator + (const Vector3D& l, const Vector3D& r)
{
    return {l.x + r.x, l.y + r.y, l.z + r.z};
}

// # v1 * v2
double operator * (const Vector3D& l, const Vector3D& r)
{
    return l.x * r.x + l.y * r.y + l.z * r.z;
}

// # v1 * a
Vector3D operator *(const Vector3D& v, const double a)
{
    return {v.x * a, v.y * a, v.z * a};
}

// # a * v1
Vector3D operator *(const double a, const Vector3D& v)
{
    return v * a;
}

// # v1 / a
Vector3D operator *(const Vector3D& v, const double a)
{
    return v * (1. / a);
}

В виде членов класса и свободных функций-друзей:

In [ ]:
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // # Animal a;
    // # ++a;
    Animal& operator++()
    {
        ++age;
        return *this;
    }
    
    // # Animal a;
    // # a++;
    Animal operator++(int)
    {
        Animal tmp = *this;
        operator++();
        return tmp;
    }

    // # Animal a;
    // # std::cout << a;
    friend std::ostream& operator <<(std::ostream& os, const Animal& animal);
};

std::ostream& operator <<(std::ostream& os, const Animal& animal)
{
    return os << animal.name << "," << animal.age;
}